In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# get all steps

from glob import glob

steps = []

# extract number from the file names
for dir in glob('/content/drive/MyDrive/bert-checkpoints/seed_0/*'):
    steps.append(int(dir.split('_')[-1]))

# sort in ascending order
steps = sorted(steps)

# should be 28+1=29
print(len(steps))

# save the steps to a local file
with open('/content/drive/MyDrive/bert-checkpoints/steps.txt', 'w') as f:
    for step in steps:
        f.write(str(step)+'\n')

29


In [5]:
steps[16:]

[800000,
 900000,
 1000000,
 1100000,
 1200000,
 1300000,
 1400000,
 1500000,
 1600000,
 1700000,
 1800000,
 1900000,
 2000000]

In [14]:
# the base directory
base_dir = '/content/drive/MyDrive/bert-checkpoints'
cfg_dir = '/content/drive/MyDrive/bert-checkpoints/bert_config.json'

# loop over 5 seeds
# for seed in [0, 1, 2, 3, 4]:
for seed in [4]:

    # create parent directory for each seed
    !cd $base_dir/pytorch_checkpoints && mkdir seed_$seed

    # loop over 29 steps
    # for step in steps:

    # first half
    # for step in steps[:15]:

    # second half
    for step in steps[15:]:

        # the directory of the tf model
        tf_dir = f'{base_dir}/seed_{seed}/step_{step}'

        # the output directory of the pytorch model
        dump_dir = f'{base_dir}/pytorch_checkpoints/seed_{seed}/step_{step}'

        # make new directory for each checkpoint
        !cd $base_dir/pytorch_checkpoints/seed_$seed && mkdir step_$step

        # copy config file to the dump directory
        !cp $cfg_dir $dump_dir/config.json

        # convert
        !transformers-cli convert --model_type bert \
          --tf_checkpoint $tf_dir/bert.ckpt \
          --config $dump_dir/config.json \
          --pytorch_dump_output $dump_dir/pytorch_model.bin

        # for testing purpose
        # break

    # for testing purpose
    # break

Streaming output truncated to the last 5000 lines.
Initialize PyTorch weight ['bert', 'embeddings', 'position_embeddings']
Initialize PyTorch weight ['bert', 'embeddings', 'token_type_embeddings']
Initialize PyTorch weight ['bert', 'embeddings', 'word_embeddings']
Initialize PyTorch weight ['bert', 'encoder', 'layer_0', 'attention', 'output', 'LayerNorm', 'beta']
Initialize PyTorch weight ['bert', 'encoder', 'layer_0', 'attention', 'output', 'LayerNorm', 'gamma']
Initialize PyTorch weight ['bert', 'encoder', 'layer_0', 'attention', 'output', 'dense', 'bias']
Initialize PyTorch weight ['bert', 'encoder', 'layer_0', 'attention', 'output', 'dense', 'kernel']
Initialize PyTorch weight ['bert', 'encoder', 'layer_0', 'attention', 'self', 'key', 'bias']
Initialize PyTorch weight ['bert', 'encoder', 'layer_0', 'attention', 'self', 'key', 'kernel']
Initialize PyTorch weight ['bert', 'encoder', 'layer_0', 'attention', 'self', 'query', 'bias']
Initialize PyTorch weight ['bert', 'encoder', 'layer_

In [30]:
# base_dir = '/content/drive/MyDrive/bert-checkpoints/seed_0/step_2000000'
# conf_dir = '/content/drive/MyDrive/bert-checkpoints'
# dump_dir = '/content/drive/MyDrive/bert-checkpoints/pytorch_checkpoints'

# /content/drive/MyDrive/bert-checkpoints/seed_0/step_0/bert.ckpt.data-00000-of-00001

# !transformers-cli convert --model_type bert \
#    --tf_checkpoint $base_dir/bert.ckpt \
#    --config $conf_dir/bert_config.json \
#    --pytorch_dump_output $dump_dir/pytorch_model.bin

In [3]:
from transformers import pipeline
from transformers import AutoModelForMaskedLM, BertTokenizerFast

In [31]:
# test if model can be loaded

model = AutoModelForMaskedLM.from_pretrained(f'/content/drive/MyDrive/bert-checkpoints/pytorch_checkpoints/seed_0/step_0')
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased') # see paper for reference

unmasker = pipeline('fill-mask', model=model, tokenizer=tokenizer)

# different mask token from RoBERTa
prior_templates = [
    '[MASK] is a [MASK].',
    '[MASK] is an [MASK].',
    '[MASK] works as a [MASK].',
    '[MASK] works as an [MASK].'
]

templates = ['[MASK] is a doctor.']

# use lower case instead
tokens = ['he', 'she']

# preview results, use fewer templates
unmasker(templates, targets=tokens)

Some weights of the model checkpoint at /content/drive/MyDrive/bert-checkpoints/pytorch_checkpoints/seed_0/step_0 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 5.237394725554623e-05,
  'token': 2016,
  'token_str': 'she',
  'sequence': 'she is a doctor.'},
 {'score': 1.906941906781867e-05,
  'token': 2002,
  'token_str': 'he',
  'sequence': 'he is a doctor.'}]